In [9]:
# 比赛链接 https://www.kaggle.com/c/facebook-v-predicting-check-ins
# 需要点击右上角参加比赛按钮后，才可以下载 数据

In [10]:
import pandas as pd

# 读取FBlocation文件夹下的train.csv
data = pd.read_csv("FBlocation/train.csv")

print(data.shape)
# 预处理：去除有缺失值的行
data = data.dropna()

# 查看数据基本信息
print("删除缺失值后数据形状：", data.shape)
print("前5行预览：")
print(data.head())



(29118021, 6)
删除缺失值后数据形状： (29118021, 6)
前5行预览：
   row_id       x       y  accuracy    time    place_id
0       0  0.7941  9.0809        54  470702  8523065625
1       1  5.9567  4.7968        13  186555  1757726713
2       2  8.3078  7.0407        74  322648  1137537235
3       3  7.3665  2.5165        65  704587  6567393236
4       4  4.0961  1.1307        31  472130  7440663949


In [11]:
# 1、缩小数据,查询数据,为了减少计算时间
data = data.query("x > 1.0 &  x < 1.25 & y > 2.5 & y < 2.75")
print(data.shape)

(17710, 6)


In [12]:
# 2、处理数据特征和标签
# 提取标签列
y = data['place_id']

# 删除不需要的列，保留特征列
X = data.drop(['row_id', 'place_id'], axis=1)

# 处理time列，转换为datetime格式并提取时间特征
X['time'] = pd.to_datetime(X['time'], unit='s')
X['day'] = X['time'].dt.day  # 提取日期中的天数
X['hour'] = X['time'].dt.hour  # 提取时间中的小时
X['weekday'] = X['time'].dt.weekday  # 提取星期几（0=周一，6=周日）

print(X)
print('-'*50)
# 删除原始time列，保留新的时间特征
X = X.drop(['time'], axis=1)

print("特征矩阵形状：", X.shape)
print("标签向量形状：", y.shape)
print("特征列：", X.columns.tolist())
print("前5行特征数据：")
print(X.head())


               x       y  accuracy                time  day  hour  weekday
600       1.2214  2.7023        17 1970-01-01 18:09:40    1    18        3
957       1.1832  2.6891        58 1970-01-10 02:11:10   10     2        5
4345      1.1935  2.6550        11 1970-01-05 15:08:02    5    15        0
4735      1.1452  2.6074        49 1970-01-06 23:03:03    6    23        1
5580      1.0089  2.7287        19 1970-01-09 11:26:50    9    11        4
...          ...     ...       ...                 ...  ...   ...      ...
29100203  1.0129  2.6775        12 1970-01-01 10:33:56    1    10        3
29108443  1.1474  2.6840        36 1970-01-07 23:22:04    7    23        2
29109993  1.0240  2.7238        62 1970-01-08 15:03:14    8    15        3
29111539  1.2032  2.6796        87 1970-01-04 00:53:41    4     0        6
29112154  1.1070  2.5419       178 1970-01-08 23:01:07    8    23        3

[17710 rows x 7 columns]
--------------------------------------------------
特征矩阵形状： (17710, 6)
标签向量

In [13]:
# 3、统计标签数据
# 统计去重后的数目
unique_places = y.nunique()
print(f"去重后的place_id数目：{unique_places}")

# 统计每个place_id的重复数目
place_counts = y.value_counts()
print(f"\n各place_id的重复数目：")
print(place_counts)

# 显示重复数目的统计信息
print(f"\n重复数目统计：")
print(f"最多重复次数：{place_counts.max()}")
print(f"最少重复次数：{place_counts.min()}")
print(f"平均重复次数：{place_counts.mean():.2f}")


去重后的place_id数目：805

各place_id的重复数目：
place_id
1097200869    1044
6683426742     894
3312463746     889
4932578245     863
5606572086     656
              ... 
7416620189       1
9268607208       1
2728953923       1
8242450582       1
7606529344       1
Name: count, Length: 805, dtype: int64

重复数目统计：
最多重复次数：1044
最少重复次数：1
平均重复次数：22.00


In [14]:
# 4、去除只去了一次的place_id的样本
# 找出重复次数大于2的place_id
places_to_keep = place_counts[place_counts > 2].index

# 过滤数据，只保留重复次数大于2的place_id对应的样本
mask = y.isin(places_to_keep)
X_filtered = X[mask]
y_filtered = y[mask]

print(f"过滤前样本数量：{len(X)}")
print(f"过滤后样本数量：{len(X_filtered)}")
print(f"过滤前place_id数量：{y.nunique()}")
print(f"过滤后place_id数量：{y_filtered.nunique()}")

# 更新X和y
X = X_filtered
y = y_filtered

print(f"\n过滤后的数据形状：")
print(f"特征矩阵形状：{X.shape}")
print(f"标签向量形状：{y.shape}")


过滤前样本数量：17710
过滤后样本数量：17086
过滤前place_id数量：805
过滤后place_id数量：295

过滤后的数据形状：
特征矩阵形状：(17086, 6)
标签向量形状：(17086,)


In [19]:
# 5、数据集划分和KNN训练评估
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler

# 划分训练集和测试集，测试集占15%
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42, stratify=y
)

print(f"训练集样本数量：{len(X_train)}")
print(f"测试集样本数量：{len(X_test)}")
print(f"训练集place_id数量：{y_train.nunique()}")
print(f"测试集place_id数量：{y_test.nunique()}")

# 数据标准化
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)  #fit_transform会在训练集计算均值和方差
X_test_scaled = scaler.transform(X_test)   #不会再次计算均值和方差，而是直接进行标准化

# 创建KNN分类器
knn = KNeighborsClassifier(n_neighbors=5)

# 训练模型
print("\n开始训练KNN模型...")
knn.fit(X_train_scaled, y_train)

# 计算准确率
accuracy = knn.score(X_test_scaled, y_test)
print(f"\nKNN模型准确率：{accuracy:.4f}")


训练集样本数量：12814
测试集样本数量：4272
训练集place_id数量：295
测试集place_id数量：295

开始训练KNN模型...

KNN模型准确率：0.4780


#  网格搜索

In [ ]:
# 使用网格搜索找到最佳的n_neighbors参数
from sklearn.model_selection import GridSearchCV

# 定义参数网格
param_grid = {
    'n_neighbors': [3, 5, 7, 9, 11],
    'weights': ['uniform', 'distance'] #uniform: 每个邻居的权重相同 distance: 根据距离进行加权，距离越近权重越大，距离越远权重越小
}

# 创建KNN分类器
knn_grid = KNeighborsClassifier()

# 创建网格搜索对象，使用5折交叉验证
grid_search = GridSearchCV(
    estimator=knn_grid,
    param_grid=param_grid,
    cv=3,
    scoring='accuracy',
    n_jobs=-1,
    verbose=1
)

# 执行网格搜索
print("开始网格搜索...")
grid_search.fit(X_train_scaled, y_train)

# 输出最佳参数和最佳得分
print(f"\n最佳参数：{grid_search.best_params_}")
print(f"最佳交叉验证得分：{grid_search.best_score_:.4f}")

# 使用最佳参数在测试集上评估
best_knn = grid_search.best_estimator_
test_accuracy = best_knn.score(X_test_scaled, y_test)
print(f"最佳模型在测试集上的准确率：{test_accuracy:.4f}")

# 显示所有参数组合的结果
print("\n所有参数组合的结果：")
results = grid_search.cv_results_
print(results)
for i in range(len(results['params'])):
    print(f"n_neighbors={results['params'][i]['n_neighbors']}: "
          f"平均得分={results['mean_test_score'][i]:.4f} "
          f"(±{results['std_test_score'][i]:.4f})")


开始网格搜索...
Fitting 3 folds for each of 10 candidates, totalling 30 fits


C:\Users\41507\AppData\Roaming\Python\Python312\site-packages\sklearn\model_selection\_split.py:805: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=3.
  warnings.warn(



最佳参数：{'n_neighbors': 9, 'weights': 'distance'}
最佳交叉验证得分：0.4774
最佳模型在测试集上的准确率：0.4953

所有参数组合的结果：
{'mean_fit_time': array([0.0070281 , 0.00831183, 0.00861367, 0.00970062, 0.00801539,
       0.00884779, 0.00907644, 0.00935157, 0.01090384, 0.01386356]), 'std_fit_time': array([0.00041075, 0.00158332, 0.00053625, 0.00105324, 0.00163843,
       0.00103237, 0.00081945, 0.00047131, 0.00231333, 0.00371354]), 'mean_score_time': array([0.14692473, 0.05114015, 0.14773734, 0.07651862, 0.14287106,
       0.08553179, 0.15035446, 0.09148113, 0.14140304, 0.0788819 ]), 'std_score_time': array([0.01285939, 0.00175867, 0.00479987, 0.00347664, 0.01267991,
       0.00491605, 0.00595782, 0.00437471, 0.00714889, 0.0027402 ]), 'param_n_neighbors': masked_array(data=[3, 3, 5, 5, 7, 7, 9, 9, 11, 11],
             mask=[False, False, False, False, False, False, False, False,
                   False, False],
       fill_value=999999), 'param_weights': masked_array(data=['uniform', 'distance', 'uniform', 'distance